In [3]:
import os
from aiohttp import ClientSession
from denokv.auth import get_database_metadata

In [4]:
session = ClientSession()
server_url = 'http://denokv:4512'
access_token = os.environ['DENOKV_PYTHON_DENOKV_ACCESS_TOKEN']

In [6]:
db_meta = await get_database_metadata(session=session, server_url=server_url, access_token=access_token)

In [7]:
db_meta

DatabaseMetadata(version=3, database_id=UUID('00000000-0000-0000-0000-000000000000'), endpoints=(EndpointInfo(url=URL('http://denokv:4512/v2'), consistency=<ConsistencyLevel.STRONG: 'strong'>),), token='hU86Re2bgI9sC26QHBwo', expires_at=datetime.datetime(2024, 9, 23, 4, 6, 59, 715983, tzinfo=datetime.timezone.utc))

In [8]:
from denokv.datapath import snapshot_read, snapshot_read_for_get

In [11]:
msg_42_read = snapshot_read_for_get(('messages', 42))

In [12]:
msg_42_read

ranges {
  start: "\002messages\000\025*"
  end: "\377"
  limit: 1
}

In [13]:
read_output = await snapshot_read(session=session, meta=db_meta, endpoint=db_meta.endpoints[0], read=msg_42_read)

In [14]:
read_output

ranges {
  values {
    key: "\002messages\000!\300D\000\000\000\000\000\000"
    value: "\377\017o"\003xxxA\002"\003WTF;"\001aI\002"\001bI\004:\004$\000\002{\001"
    encoding: VE_V8
    versionstamp: "\000\000\000\000\000\000\000.\000\000"
  }
}
read_is_strongly_consistent: true
status: SR_SUCCESS

In [20]:
read_output.ranges[0].values[0].value

b'\xff\x0fo"\x03xxxA\x02"\x03WTF;"\x01aI\x02"\x01bI\x04:\x04$\x00\x02{\x01'

In [21]:
import v8serialize

In [22]:
v8serialize.loads(read_output.ranges[0].values[0].value)

JSObject(
  xxx=JSArray([
    'WTF',
    JSMap({
      'a': 1,
      'b': 2,
    }),
  ]),
)